<a href="https://colab.research.google.com/github/AyanChawla/Coursera_Capstone/blob/master/Neighbourhood_in_TorontoPart2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project

Getting data from Wikipedia and preproccessing it.

In [0]:
from bs4 import BeautifulSoup as bsoup
from urllib.request import urlopen as uReq
import requests
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np

Using Requests module to get wikipedia page

In [0]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

Using bs4 to scrape the Data


In [0]:
results=bsoup(r.text,"html.parser")

In [57]:
type(results)

bs4.BeautifulSoup

From inspecting we get that it is inside the table element

In [0]:
table = results.table
table_rows=table.find_all('tr')

In [59]:
table_rows[0]

<tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>

In [60]:
table_rows[0].text

'\nPostal Code\n\nBorough\n\nNeighborhood\n'

In [61]:
nrows=len(table_rows)
nrows

181

There are 181 rows

Lets make a DF

In [62]:
table_rows[10].text.split('\n')

['', 'M1B', '', 'Scarborough', '', 'Malvern / Rouge', '']

It shows that index 1 has Postal code, 3 has Borough and 5 has neighbourhood

In [63]:
rows=[]
n=1
while n < nrows :
    Postcode=table_rows[n].text.split('\n')[1]
    Borough=table_rows[n].text.split('\n')[3]
    Neighborhood=table_rows[n].text.split('\n')[5]
    rows.append((Postcode, Borough,Neighborhood))
    n=n+1

dataset=pd.DataFrame(rows, columns=['PostalCode', 'Borough', 'Neighbourhood'])
dataset.head(5)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [64]:
dataset.shape

(180, 3)

It has 180 rows and 3 columns

In [65]:
dataset.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

Total 77 has not assigned so now we will delete 180-77=103

In [0]:
dataset.drop(dataset[dataset['Borough']=='Not assigned'].index,inplace=True)

In [67]:
dataset.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

It has deleted those

Now lets replace the neighbourhood column






In [0]:
dataset.loc[dataset['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = dataset['Borough']

In [69]:
dataset.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [70]:
dataset.shape

(103, 3)

Now lets combine the neighbourhoods having same postal codes

In [71]:
dataset.PostalCode.value_counts().count()

103

It is already combined with it

In [72]:
dataset.shape

(103, 3)

End of part1

# Part 2 Getting longitudes and Latitudes

In [82]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('M1B, Toronto, Ontario')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: ignored

It take very long time to fetch one detail

In [83]:
url='http://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Lets rename the column to merge it

In [0]:
df_pcodes.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [86]:
df_pcodes

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [87]:
neighborhoods=pd.merge(dataset,df_pcodes, how='right', on = 'PostalCode')
neighborhoods.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,Garden District / Ryerson,43.657162,-79.378937


Lets sort the values of postal code

In [89]:
neighborhoods=neighborhoods.sort_values('PostalCode')
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
18,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [90]:
neighborhoods.shape

(103, 5)

# end of part 2